In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms

print("done")

done


In [8]:
def view_classify(img, ps):

    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

In [10]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [11]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/home/umut/Documents/GitHub/Strive_School/Computer_Vision_(Module_6)/Day_4_ Kernels_callbacks_and_thresholding/MNIST_data', train=True, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/home/umut/Documents/GitHub/Strive_School/Computer_Vision_(Module_6)/Day_4_ Kernels_callbacks_and_thresholding/MNIST_data', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [35]:
class Network(nn.Module):
    
    # Defining the layers, 128, 64, 10 units each
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Network()
model

Network(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [36]:
network = Network()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [37]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), '/home/umut/Documents/GitHub/Strive_School/Computer_Vision_(Module_6)/Day_7_ Template_matching_corners_and_Haar_cascades/Modules/model.pth')
      torch.save(optimizer.state_dict(), '/home/umut/Documents/GitHub/Strive_School/Computer_Vision_(Module_6)/Day_7_ Template_matching_corners_and_Haar_cascades/Modules/optimizer.pth')
      


In [41]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [42]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

<ipython-input-35-bd35754d1a27>:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.414373
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.254169
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.281210
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.223230
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.163231
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.196350
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.173330
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.198954
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.251421
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.256380
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.167106
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.310119
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.447991
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.330288
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.448821
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.240600
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0

In [28]:
def to_mnist(img):
    img = cv2.blur(img, (11, 11))
    _, img = cv2.threshold(img, 150, 255, cv2.THRESH_TRUNC)
    img = cv2.bitwise_not(img)
    kernel = np.ones((11, 11), np.uint8)
    # img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=1)
    img = cv2.dilate(img, kernel, iterations=1)
    _, img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    # kernel_er = np.ones((7, 7), np.uint8)
    # img = cv2.erode(img, kernel_er, iterations=1)
    img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_LANCZOS4)

    return img

def to_tensor(arr: np.ndarray):
    tens = torch.from_numpy(arr).float()
    tens = F.normalize(tens)
    #tens = F.normalize(tens,0.137,0.3081)
    tens = tens.reshape(-1 ,784)
    return tens


"""
transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                                 """
                                 

'\ntransform=torchvision.transforms.Compose([\n                               torchvision.transforms.ToTensor(),\n                               torchvision.transforms.Normalize(\n                                 (0.1307,), (0.3081,))\n                                 '

In [47]:
three = cv2.imread("/home/umut/Documents/GitHub/Strive_School/Computer_Vision_(Module_6)/Day_4_ Kernels_callbacks_and_thresholding/img/3.jpg")
three=to_mnist(three)
three = to_tensor(three)

outputs = network(three)

_, predicted = torch.max(outputs.data,1)
predicted
outputs.shape
print(outputs.shape)
print(predicted)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [10, 1, 5, 5], but got 2-dimensional input of size [3, 784] instead

In [ ]:
def findNextCellToFill(grid, i, j):
        for x in range(i,9):
                for y in range(j,9):
                        if grid[x][y] == 0:
                                return x,y
        for x in range(0,9):
                for y in range(0,9):
                        if grid[x][y] == 0:
                                return x,y
        return -1,-1

def isValid(grid, i, j, e):
        rowOk = all([e != grid[i][x] for x in range(9)])
        if rowOk:
                columnOk = all([e != grid[x][j] for x in range(9)])
                if columnOk:
                        # finding the top left x,y co-ordinates of the section containing the i,j cell
                        secTopX, secTopY = 3 *(i//3), 3 *(j//3) #floored quotient should be used here. 
                        for x in range(secTopX, secTopX+3):
                                for y in range(secTopY, secTopY+3):
                                        if grid[x][y] == e:
                                                return False
                        return True
        return False

def solveSudoku(grid, i=0, j=0):
        i,j = findNextCellToFill(grid, i, j)
        if i == -1:
                return True
        for e in range(1,10):
                if isValid(grid,i,j,e):
                        grid[i][j] = e
                        if solveSudoku(grid, i, j):
                                return True
                        # Undo the current cell for backtracking
                        grid[i][j] = 0
        return False